Este projeto destina-se a criar um modelo de machine learning capaz de classificar as tags das perguntas do site stackoverflow.

# Imports

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer

# Coleta de dados

In [2]:
perguntas = pd.read_csv('https://raw.githubusercontent.com/alura-cursos/alura_classificacao_multilabel/master/dataset/stackoverflow_perguntas.csv')

# Análise Exploratória

In [3]:
# sample dos dados para investigação inicial
perguntas.sample(10)

,Perguntas,Tags
5027,"O plugin Select2 funciona normalmente, o único...",jquery
3671,Criei um script básico para exibir/ocultar um ...,jquery
4370,Eu tenho um campo number onde preciso que seja...,jquery html
5092,Bom dia Galera. Estou consumindo uma api exte...,node.js
4984,"Galera, preciso de ajuda com o Angular 4... pr...",angular
771,Tenho um texto e quero pegar as posições inici...,jquery
1282,"Olá, estou tendo um problema com o JavaScript ...",html
4585,Como realizar o procedimento de captura de ima...,html
4687,"Tenho uns campo campos imputs, com valores. Fi...",jquery
670,Eu preciso preciso criar div's com dados de du...,jquery html


Nas tags, há mais que uma característica para algumas perguntas, trata-se então de uma coluna multilabel.

In [4]:
print(len(perguntas))

5408


Há 5408 perguntas no dataframe.

## Quantas combinações de tags há no dataframe?

In [5]:
perguntas['Tags'].unique()

array(['node.js', 'jquery', 'html', 'html angular ', 'html ', 'angular',
       'angular ', 'jquery html  ', 'jquery ', 'jquery html',
       'jquery html ', 'html angular', 'angular node.js ', 'html  ',
       'jquery html angular', 'node.js ', 'html jquery', 'html jquery ',
       'jquery angular  ', 'html node.js', 'jquery  ', 'angular node.js',
       'jquery angular', 'html node.js ', 'jquery node.js ', 'angular  ',
       'jquery angular ', 'jquery html angular ', 'node.js html ',
       ' node.js', 'node.js html', 'html angular  ', 'jquery node.js',
       'angular html', 'html angular  node.js', 'jquery html node.js',
       'html angular node.js'], dtype=object)

In [6]:
perguntas['Tags'].nunique()

37

Há 37 combinações de tags, onde muitas delas, há as mesmas tecnologias a serem consideradas pois espaços tornam strings diferentes umas das outras ou tecnologias invertidas, ainda que possuam o mesmo conteúdo.

# Transformando strings em labels numéricas

In [7]:
# cria lista vazia que armazenará as labels
lista_de_tags = list()
# seleciona as tags que estão na array 
for tags in perguntas['Tags'].unique():
    # divide as strings em strings únicas 
    for tag in tags.split():
        # se alguma dessas strings não estiver na lista vazia, coloque-a dentro dela
        if tag not in lista_de_tags:
            lista_de_tags.append(tag)

# mostra a lista
print(lista_de_tags)

['node.js', 'jquery', 'html', 'angular']


In [8]:
# # cria lista de dummy node_js
# node_js = list()
# # para cada uma das linhas, se houver a string 'node.js', 1, se não, 0
# for linha_tag in perguntas['Tags']:
#     if 'node.js' in linha_tag:
#         node_js.append(1)
#     else:
#         node_js.append(0)
# # criando coluna node.js no dataframe
# perguntas['node_js'] = node_js

Seria necessário fazer isso para cada uma das tags, portanto faz-se necessário a criação de uma função.

In [9]:
# função que cria colunas dummies para cada tag
def nova_coluna(lista_tags, dataframe, nome_tags):
    # lendo a tag na lista de tags obtidas na iteração anterior
    for tag in lista_tags:
        # lista vazia que armazenará os dados da coluna
        coluna = list()
        # lendo as linhas no dataframe que possuem as tags
        for linha_tag in dataframe[nome_tags]:
            # caso a tag esteja nessa linha, 1, se não, 0
            if tag in linha_tag:
                coluna.append(1)
            else:
                coluna.append(0)
        # após criada a lista com as dummies, insere-a como coluna do dataframe
        dataframe[tag] = coluna

In [10]:
# criando colunas dummies com as tags
nova_coluna(lista_tags = lista_de_tags, dataframe = perguntas, nome_tags = 'Tags')

In [11]:
perguntas.head()

,Perguntas,Tags,node.js,jquery,html,angular
0,Possuo um projeto Node.js porém preciso criar ...,node.js,1,0,0,0
1,"Gostaria de fazer testes unitários no Node.js,...",node.js,1,0,0,0
2,Como inverter a ordem com que o jQuery itera u...,jquery,0,1,0,0
3,Eu tenho uma página onde pretendo utilizar um ...,html,0,0,1,0
4,Como exibir os dados retornados do FireStore e...,html angular,0,0,1,1


In [12]:
# criando lista de tuplas com as 4 colunas target
lista_zip_tags = list(zip(perguntas[lista_de_tags[0]],
                     perguntas[lista_de_tags[1]],
                     perguntas[lista_de_tags[2]],
                     perguntas[lista_de_tags[3]]))

# criando coluna no dataframe
perguntas['todas_tags'] = lista_zip_tags

In [13]:
list1 = [1, 2]
list2 = [3, 4]
list3 = list(zip(list1, list2))
print(list3)

[(1, 3), (2, 4)]


In [14]:
perguntas.head()

,Perguntas,Tags,node.js,jquery,html,angular,todas_tags
0,Possuo um projeto Node.js porém preciso criar ...,node.js,1,0,0,0,"(1, 0, 0, 0)"
1,"Gostaria de fazer testes unitários no Node.js,...",node.js,1,0,0,0,"(1, 0, 0, 0)"
2,Como inverter a ordem com que o jQuery itera u...,jquery,0,1,0,0,"(0, 1, 0, 0)"
3,Eu tenho uma página onde pretendo utilizar um ...,html,0,0,1,0,"(0, 0, 1, 0)"
4,Como exibir os dados retornados do FireStore e...,html angular,0,0,1,1,"(0, 0, 1, 1)"


In [15]:
# dividindo o dataframe em dataframes de treino e teste
perguntas_treino, perguntas_test, tags_treino, tags_teste = train_test_split(perguntas['Perguntas'], perguntas['todas_tags'],
                                                                            test_size = 0.3, random_state = 123)               

# Vetorização

In [16]:
# instanciando o vetorizador
# max_features: parâmetro que limita o número de features do vetor
# max_df: desconsidera palavras com muita frequência
vetorizar = TfidfVectorizer(max_features = 5000, max_df = 0.85)

In [17]:
# treinando o vetorizador
vetorizar.fit(perguntas['Perguntas'])

# transformando vetores
perguntas_treino_tfidf = vetorizar.transform(perguntas_treino)
perguntas_test_tfidf = vetorizar.transform(perguntas_test)

In [18]:
print(perguntas_treino_tfidf.shape)
print(perguntas_test_tfidf.shape)

(3785, 5000)
(1623, 5000)
